In [1]:
import numpy as np
import pandas as pd
from gurobipy import GRB
import gurobipy as gp
import random
import networkx as nx
import math
import time

In [2]:
NodeCount = 200
GraphDensityCoefficient = 0.5
MaxCostArc = 20
RequiredInfo = 20
CommodityCount = 5
MaxDemandCommodity = 20
MaxCapacityArc = 40

random.seed(10)

N = [i+1 for i in range(NodeCount)]
Commodities = [i+1 for i in range(CommodityCount)]
NodeNodeAdjacencyMatrix = np.zeros((NodeCount,NodeCount))


ArcProbability = np.zeros((NodeCount,NodeCount))

for i in range(NodeCount-1):
    for j in range(i+1,NodeCount):
        ArcProbability[i,j] = random.uniform(0,1)
        if ArcProbability[i,j] > GraphDensityCoefficient:
            
            NodeNodeAdjacencyMatrix[i,j] = 1
            
    if sum(NodeNodeAdjacencyMatrix[i]) == 0:
        
        NodeNodeAdjacencyMatrix[i,NodeCount-1] = 1

ArcCount = int(sum(sum(NodeNodeAdjacencyMatrix)))



In [3]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
ArcCount    

9870

In [5]:
# G = nx.from_numpy_array(NodeNodeAdjacencyMatrix,create_using=nx.DiGraph())
# nx.draw_circular(G,arrows=True)

In [6]:
NodeNodeAdjacencyList = {i:[] for i in N}
CostDict = {}
CapacityDict = {}
ArcDict = {i:0 for i in range(ArcCount)}
InfoDict = {i:0 for i in range(ArcCount)}

ArcCounter = 0

for i in range(NodeCount):
    for j in range(NodeCount):
        if NodeNodeAdjacencyMatrix[i][j] == 1:
            NodeNodeAdjacencyList[i+1].append(j+1)
            CostDict[ArcCounter]= random.randint(1,MaxCostArc)
            CapacityDict[ArcCounter] = random.randint(MaxCapacityArc/2,MaxCapacityArc)
            InfoDict[ArcCounter] = random.randint(0,RequiredInfo/4)
            ArcDict[ArcCounter] = (i+1,j+1)
            ArcCounter += 1

In [7]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (1, 10),
 6: (1, 13),
 7: (1, 14),
 8: (1, 16),
 9: (1, 17),
 10: (1, 20),
 11: (1, 22),
 12: (1, 23),
 13: (1, 25),
 14: (1, 26),
 15: (1, 27),
 16: (1, 28),
 17: (1, 32),
 18: (1, 35),
 19: (1, 37),
 20: (1, 40),
 21: (1, 41),
 22: (1, 47),
 23: (1, 48),
 24: (1, 49),
 25: (1, 50),
 26: (1, 52),
 27: (1, 54),
 28: (1, 56),
 29: (1, 59),
 30: (1, 60),
 31: (1, 65),
 32: (1, 70),
 33: (1, 71),
 34: (1, 72),
 35: (1, 77),
 36: (1, 82),
 37: (1, 84),
 38: (1, 85),
 39: (1, 87),
 40: (1, 88),
 41: (1, 89),
 42: (1, 90),
 43: (1, 96),
 44: (1, 97),
 45: (1, 101),
 46: (1, 105),
 47: (1, 110),
 48: (1, 114),
 49: (1, 116),
 50: (1, 117),
 51: (1, 118),
 52: (1, 120),
 53: (1, 121),
 54: (1, 124),
 55: (1, 125),
 56: (1, 126),
 57: (1, 127),
 58: (1, 128),
 59: (1, 130),
 60: (1, 131),
 61: (1, 133),
 62: (1, 134),
 63: (1, 135),
 64: (1, 137),
 65: (1, 138),
 66: (1, 139),
 67: (1, 140),
 68: (1, 142),
 69: (1, 144),
 70: (1, 1

In [8]:
NodeNodeAdjacencyList

{1: [2,
  4,
  6,
  7,
  8,
  10,
  13,
  14,
  16,
  17,
  20,
  22,
  23,
  25,
  26,
  27,
  28,
  32,
  35,
  37,
  40,
  41,
  47,
  48,
  49,
  50,
  52,
  54,
  56,
  59,
  60,
  65,
  70,
  71,
  72,
  77,
  82,
  84,
  85,
  87,
  88,
  89,
  90,
  96,
  97,
  101,
  105,
  110,
  114,
  116,
  117,
  118,
  120,
  121,
  124,
  125,
  126,
  127,
  128,
  130,
  131,
  133,
  134,
  135,
  137,
  138,
  139,
  140,
  142,
  144,
  145,
  152,
  153,
  155,
  156,
  157,
  159,
  160,
  163,
  164,
  166,
  167,
  168,
  169,
  176,
  177,
  179,
  180,
  181,
  185,
  190,
  191,
  192,
  194,
  195,
  196,
  197,
  199,
  200],
 2: [6,
  7,
  12,
  13,
  15,
  16,
  18,
  20,
  21,
  24,
  25,
  26,
  27,
  28,
  29,
  32,
  34,
  35,
  37,
  40,
  41,
  42,
  44,
  45,
  48,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  60,
  65,
  67,
  68,
  69,
  70,
  76,
  79,
  82,
  84,
  90,
  92,
  93,
  95,
  96,
  98,
  99,
  102,
  105,
  108,
  109,
  111,
  112,
  113,
  114,
  

In [9]:
InfoDict

{0: 3,
 1: 5,
 2: 5,
 3: 5,
 4: 3,
 5: 4,
 6: 1,
 7: 1,
 8: 4,
 9: 4,
 10: 2,
 11: 1,
 12: 3,
 13: 1,
 14: 3,
 15: 5,
 16: 0,
 17: 1,
 18: 5,
 19: 5,
 20: 0,
 21: 2,
 22: 5,
 23: 1,
 24: 5,
 25: 3,
 26: 3,
 27: 3,
 28: 4,
 29: 4,
 30: 2,
 31: 4,
 32: 2,
 33: 2,
 34: 2,
 35: 4,
 36: 2,
 37: 0,
 38: 3,
 39: 0,
 40: 3,
 41: 0,
 42: 1,
 43: 2,
 44: 5,
 45: 1,
 46: 1,
 47: 4,
 48: 0,
 49: 5,
 50: 5,
 51: 1,
 52: 2,
 53: 3,
 54: 0,
 55: 1,
 56: 4,
 57: 1,
 58: 5,
 59: 3,
 60: 4,
 61: 4,
 62: 1,
 63: 3,
 64: 2,
 65: 3,
 66: 0,
 67: 3,
 68: 1,
 69: 2,
 70: 0,
 71: 4,
 72: 2,
 73: 3,
 74: 0,
 75: 2,
 76: 5,
 77: 2,
 78: 2,
 79: 4,
 80: 1,
 81: 0,
 82: 4,
 83: 5,
 84: 1,
 85: 4,
 86: 1,
 87: 0,
 88: 3,
 89: 4,
 90: 1,
 91: 1,
 92: 3,
 93: 3,
 94: 2,
 95: 1,
 96: 3,
 97: 0,
 98: 3,
 99: 5,
 100: 2,
 101: 3,
 102: 2,
 103: 0,
 104: 3,
 105: 0,
 106: 2,
 107: 5,
 108: 5,
 109: 4,
 110: 4,
 111: 1,
 112: 0,
 113: 3,
 114: 3,
 115: 3,
 116: 3,
 117: 4,
 118: 5,
 119: 4,
 120: 0,
 121: 0,
 122: 2,
 12

In [10]:
CostDict

{0: 12,
 1: 11,
 2: 18,
 3: 19,
 4: 18,
 5: 19,
 6: 10,
 7: 8,
 8: 10,
 9: 19,
 10: 4,
 11: 2,
 12: 20,
 13: 11,
 14: 11,
 15: 5,
 16: 19,
 17: 1,
 18: 4,
 19: 19,
 20: 6,
 21: 4,
 22: 18,
 23: 8,
 24: 16,
 25: 14,
 26: 17,
 27: 17,
 28: 13,
 29: 13,
 30: 14,
 31: 14,
 32: 11,
 33: 18,
 34: 18,
 35: 17,
 36: 19,
 37: 19,
 38: 7,
 39: 14,
 40: 8,
 41: 10,
 42: 8,
 43: 10,
 44: 6,
 45: 1,
 46: 11,
 47: 6,
 48: 12,
 49: 19,
 50: 5,
 51: 10,
 52: 4,
 53: 8,
 54: 1,
 55: 14,
 56: 10,
 57: 4,
 58: 20,
 59: 6,
 60: 9,
 61: 2,
 62: 18,
 63: 5,
 64: 11,
 65: 11,
 66: 15,
 67: 2,
 68: 9,
 69: 15,
 70: 19,
 71: 14,
 72: 15,
 73: 15,
 74: 7,
 75: 18,
 76: 7,
 77: 14,
 78: 6,
 79: 3,
 80: 19,
 81: 9,
 82: 12,
 83: 20,
 84: 3,
 85: 14,
 86: 16,
 87: 19,
 88: 8,
 89: 19,
 90: 3,
 91: 20,
 92: 16,
 93: 12,
 94: 13,
 95: 1,
 96: 3,
 97: 1,
 98: 17,
 99: 1,
 100: 14,
 101: 19,
 102: 7,
 103: 5,
 104: 13,
 105: 1,
 106: 20,
 107: 4,
 108: 8,
 109: 5,
 110: 5,
 111: 15,
 112: 17,
 113: 19,
 114: 15,
 115:

In [11]:
CapacityDict

{0: 21,
 1: 25,
 2: 31,
 3: 25,
 4: 35,
 5: 26,
 6: 20,
 7: 21,
 8: 31,
 9: 22,
 10: 37,
 11: 28,
 12: 22,
 13: 24,
 14: 39,
 15: 31,
 16: 28,
 17: 24,
 18: 24,
 19: 28,
 20: 30,
 21: 36,
 22: 21,
 23: 25,
 24: 22,
 25: 20,
 26: 36,
 27: 27,
 28: 34,
 29: 21,
 30: 32,
 31: 40,
 32: 38,
 33: 36,
 34: 21,
 35: 36,
 36: 26,
 37: 33,
 38: 33,
 39: 39,
 40: 35,
 41: 35,
 42: 40,
 43: 37,
 44: 33,
 45: 30,
 46: 40,
 47: 38,
 48: 22,
 49: 35,
 50: 22,
 51: 22,
 52: 35,
 53: 27,
 54: 28,
 55: 36,
 56: 33,
 57: 24,
 58: 37,
 59: 20,
 60: 39,
 61: 37,
 62: 30,
 63: 22,
 64: 27,
 65: 40,
 66: 39,
 67: 30,
 68: 39,
 69: 21,
 70: 35,
 71: 37,
 72: 21,
 73: 25,
 74: 38,
 75: 27,
 76: 27,
 77: 38,
 78: 20,
 79: 25,
 80: 26,
 81: 23,
 82: 28,
 83: 37,
 84: 21,
 85: 25,
 86: 28,
 87: 38,
 88: 23,
 89: 21,
 90: 24,
 91: 23,
 92: 23,
 93: 33,
 94: 28,
 95: 22,
 96: 21,
 97: 23,
 98: 21,
 99: 30,
 100: 34,
 101: 22,
 102: 35,
 103: 38,
 104: 36,
 105: 26,
 106: 39,
 107: 27,
 108: 27,
 109: 34,
 110: 38,


In [12]:
B_k = [random.randint(1,MaxDemandCommodity) for i in range(CommodityCount)]
B_k

[15, 5, 9, 9, 11]

In [13]:
Origin = [random.randint(1,math.floor(NodeCount/2)) for i in range(CommodityCount)]
Destination = [random.randint(math.floor(NodeCount/2)+1,NodeCount) for i in range(CommodityCount)]

In [14]:
Origin

[50, 11, 19, 45, 34]

In [15]:
Destination

[152, 115, 158, 115, 150]

In [16]:
def Reachable(visited, graph, node): 
    visited.append(node)
    queue.append(node)
    Reachable = []

    while queue:          
        m = queue.pop(0) 
#         print (m, end = " ") 
        Reachable.append(m)

        for neighbour in graph[m]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return Reachable

In [17]:
for i in range(len(Origin)):
    visited = [] # List for visited nodes.
    queue = []     #Initialize a queue
    ReachableList = Reachable(visited,NodeNodeAdjacencyList,Origin[i])
#     print(i,Origin[i],Destination[i],ReachableList)
    k = len(ReachableList)

# #             break
    if Destination[i] not in ReachableList:
        for j in range(k):
            if ReachableList[j]>Origin[i]:
                Destination[i] = ReachableList[j]
                
        

In [18]:
Destination

[152, 115, 158, 115, 150]

In [19]:
B_k_Dict = {i:[] for i in Commodities}
Origin_Dict = {i:[] for i in Commodities}
Destination_Dict = {i:[] for i in Commodities}
for i in range(CommodityCount):
    B_k_Dict[i+1]=B_k[i]
    Origin_Dict[i+1] = Origin[i]
    Destination_Dict[i+1] = Destination[i]

In [20]:
NodeNodeAdjacencyListWithDummyArcs = {i:[] for i in N}
# PathCost = {}
# CapacityListInitial = {}
InitialHighCost = MaxCostArc*ArcCount + 1
InitialHighCapacity = MaxDemandCommodity*ArcCount + 1

for i in range(len(Origin)):

    NodeNodeAdjacencyListWithDummyArcs[Origin[i]].append(Destination[i])



In [21]:
PathSetForCommodity = {i:[] for i in Commodities}
Paths = [i for i in range(CommodityCount)]

for i in range(CommodityCount):
    PathSetForCommodity[i+1].append(Paths[i])
PathSetForCommodity

{1: [0], 2: [1], 3: [2], 4: [3], 5: [4]}

In [22]:
Paths

[0, 1, 2, 3, 4]

In [23]:
PathDict = {i:[] for i in Paths}
NodesInPath = {i:[] for i in Paths}
for i in Paths:
    PathDict[i].append((Origin[i],Destination[i]))
    NodesInPath[i].append(Origin[i])
    NodesInPath[i].append(Destination[i])
PathDict

{0: [(50, 152)],
 1: [(11, 115)],
 2: [(19, 158)],
 3: [(45, 115)],
 4: [(34, 150)]}

In [24]:
NodesInPath

{0: [50, 152], 1: [11, 115], 2: [19, 158], 3: [45, 115], 4: [34, 150]}

In [25]:
PathCost = {p:InitialHighCost for p in Paths}
PathInfo = {p:RequiredInfo for p in Paths}
for i in Paths:
    for j in range(ArcCount):
        if list(ArcDict.values())[j] == (PathDict[i][0]):
            ArcIndex = list(ArcDict.values())[j]
            PathCost[i] = CostDict[j]
            PathInfo[i] = InfoDict[j]
# for i in Paths:
#     if PathCost[i] == InitialHighCost:
#         PathInfo[i] = RequiredInfo

        
#         else:
#             PathCost[i] = InitialHighCost
    
Counter = 0

for i in N:
    for j in N:
        if (i in Origin) and (j in Destination):
            
            if NodeNodeAdjacencyMatrix[i-1,j-1] == 0:
                CapacityDict[ArcCount+Counter] = InitialHighCapacity
                CostDict[ArcCount+Counter] = InitialHighCost
                InfoDict[ArcCount+Counter] = RequiredInfo
                NodeNodeAdjacencyMatrix[i-1,j-1] = 1
                ArcDict[ArcCount+Counter] = (i,j)
                
                Counter += 1
                
#             else:
#                 PathCost[(i,j)] = CostList[(i,j)]
ArcCountWithDummyArcs = ArcCount + Counter

In [26]:
PathInfo

{0: 20, 1: 5, 2: 20, 3: 20, 4: 0}

In [27]:
PathCost

{0: 197401, 1: 19, 2: 197401, 3: 197401, 4: 18}

In [28]:
CapacityDict

{0: 21,
 1: 25,
 2: 31,
 3: 25,
 4: 35,
 5: 26,
 6: 20,
 7: 21,
 8: 31,
 9: 22,
 10: 37,
 11: 28,
 12: 22,
 13: 24,
 14: 39,
 15: 31,
 16: 28,
 17: 24,
 18: 24,
 19: 28,
 20: 30,
 21: 36,
 22: 21,
 23: 25,
 24: 22,
 25: 20,
 26: 36,
 27: 27,
 28: 34,
 29: 21,
 30: 32,
 31: 40,
 32: 38,
 33: 36,
 34: 21,
 35: 36,
 36: 26,
 37: 33,
 38: 33,
 39: 39,
 40: 35,
 41: 35,
 42: 40,
 43: 37,
 44: 33,
 45: 30,
 46: 40,
 47: 38,
 48: 22,
 49: 35,
 50: 22,
 51: 22,
 52: 35,
 53: 27,
 54: 28,
 55: 36,
 56: 33,
 57: 24,
 58: 37,
 59: 20,
 60: 39,
 61: 37,
 62: 30,
 63: 22,
 64: 27,
 65: 40,
 66: 39,
 67: 30,
 68: 39,
 69: 21,
 70: 35,
 71: 37,
 72: 21,
 73: 25,
 74: 38,
 75: 27,
 76: 27,
 77: 38,
 78: 20,
 79: 25,
 80: 26,
 81: 23,
 82: 28,
 83: 37,
 84: 21,
 85: 25,
 86: 28,
 87: 38,
 88: 23,
 89: 21,
 90: 24,
 91: 23,
 92: 23,
 93: 33,
 94: 28,
 95: 22,
 96: 21,
 97: 23,
 98: 21,
 99: 30,
 100: 34,
 101: 22,
 102: 35,
 103: 38,
 104: 36,
 105: 26,
 106: 39,
 107: 27,
 108: 27,
 109: 34,
 110: 38,


In [29]:
InfoDict

{0: 3,
 1: 5,
 2: 5,
 3: 5,
 4: 3,
 5: 4,
 6: 1,
 7: 1,
 8: 4,
 9: 4,
 10: 2,
 11: 1,
 12: 3,
 13: 1,
 14: 3,
 15: 5,
 16: 0,
 17: 1,
 18: 5,
 19: 5,
 20: 0,
 21: 2,
 22: 5,
 23: 1,
 24: 5,
 25: 3,
 26: 3,
 27: 3,
 28: 4,
 29: 4,
 30: 2,
 31: 4,
 32: 2,
 33: 2,
 34: 2,
 35: 4,
 36: 2,
 37: 0,
 38: 3,
 39: 0,
 40: 3,
 41: 0,
 42: 1,
 43: 2,
 44: 5,
 45: 1,
 46: 1,
 47: 4,
 48: 0,
 49: 5,
 50: 5,
 51: 1,
 52: 2,
 53: 3,
 54: 0,
 55: 1,
 56: 4,
 57: 1,
 58: 5,
 59: 3,
 60: 4,
 61: 4,
 62: 1,
 63: 3,
 64: 2,
 65: 3,
 66: 0,
 67: 3,
 68: 1,
 69: 2,
 70: 0,
 71: 4,
 72: 2,
 73: 3,
 74: 0,
 75: 2,
 76: 5,
 77: 2,
 78: 2,
 79: 4,
 80: 1,
 81: 0,
 82: 4,
 83: 5,
 84: 1,
 85: 4,
 86: 1,
 87: 0,
 88: 3,
 89: 4,
 90: 1,
 91: 1,
 92: 3,
 93: 3,
 94: 2,
 95: 1,
 96: 3,
 97: 0,
 98: 3,
 99: 5,
 100: 2,
 101: 3,
 102: 2,
 103: 0,
 104: 3,
 105: 0,
 106: 2,
 107: 5,
 108: 5,
 109: 4,
 110: 4,
 111: 1,
 112: 0,
 113: 3,
 114: 3,
 115: 3,
 116: 3,
 117: 4,
 118: 5,
 119: 4,
 120: 0,
 121: 0,
 122: 2,
 12

In [30]:
CostDict

{0: 12,
 1: 11,
 2: 18,
 3: 19,
 4: 18,
 5: 19,
 6: 10,
 7: 8,
 8: 10,
 9: 19,
 10: 4,
 11: 2,
 12: 20,
 13: 11,
 14: 11,
 15: 5,
 16: 19,
 17: 1,
 18: 4,
 19: 19,
 20: 6,
 21: 4,
 22: 18,
 23: 8,
 24: 16,
 25: 14,
 26: 17,
 27: 17,
 28: 13,
 29: 13,
 30: 14,
 31: 14,
 32: 11,
 33: 18,
 34: 18,
 35: 17,
 36: 19,
 37: 19,
 38: 7,
 39: 14,
 40: 8,
 41: 10,
 42: 8,
 43: 10,
 44: 6,
 45: 1,
 46: 11,
 47: 6,
 48: 12,
 49: 19,
 50: 5,
 51: 10,
 52: 4,
 53: 8,
 54: 1,
 55: 14,
 56: 10,
 57: 4,
 58: 20,
 59: 6,
 60: 9,
 61: 2,
 62: 18,
 63: 5,
 64: 11,
 65: 11,
 66: 15,
 67: 2,
 68: 9,
 69: 15,
 70: 19,
 71: 14,
 72: 15,
 73: 15,
 74: 7,
 75: 18,
 76: 7,
 77: 14,
 78: 6,
 79: 3,
 80: 19,
 81: 9,
 82: 12,
 83: 20,
 84: 3,
 85: 14,
 86: 16,
 87: 19,
 88: 8,
 89: 19,
 90: 3,
 91: 20,
 92: 16,
 93: 12,
 94: 13,
 95: 1,
 96: 3,
 97: 1,
 98: 17,
 99: 1,
 100: 14,
 101: 19,
 102: 7,
 103: 5,
 104: 13,
 105: 1,
 106: 20,
 107: 4,
 108: 8,
 109: 5,
 110: 5,
 111: 15,
 112: 17,
 113: 19,
 114: 15,
 115:

In [31]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (1, 10),
 6: (1, 13),
 7: (1, 14),
 8: (1, 16),
 9: (1, 17),
 10: (1, 20),
 11: (1, 22),
 12: (1, 23),
 13: (1, 25),
 14: (1, 26),
 15: (1, 27),
 16: (1, 28),
 17: (1, 32),
 18: (1, 35),
 19: (1, 37),
 20: (1, 40),
 21: (1, 41),
 22: (1, 47),
 23: (1, 48),
 24: (1, 49),
 25: (1, 50),
 26: (1, 52),
 27: (1, 54),
 28: (1, 56),
 29: (1, 59),
 30: (1, 60),
 31: (1, 65),
 32: (1, 70),
 33: (1, 71),
 34: (1, 72),
 35: (1, 77),
 36: (1, 82),
 37: (1, 84),
 38: (1, 85),
 39: (1, 87),
 40: (1, 88),
 41: (1, 89),
 42: (1, 90),
 43: (1, 96),
 44: (1, 97),
 45: (1, 101),
 46: (1, 105),
 47: (1, 110),
 48: (1, 114),
 49: (1, 116),
 50: (1, 117),
 51: (1, 118),
 52: (1, 120),
 53: (1, 121),
 54: (1, 124),
 55: (1, 125),
 56: (1, 126),
 57: (1, 127),
 58: (1, 128),
 59: (1, 130),
 60: (1, 131),
 61: (1, 133),
 62: (1, 134),
 63: (1, 135),
 64: (1, 137),
 65: (1, 138),
 66: (1, 139),
 67: (1, 140),
 68: (1, 142),
 69: (1, 144),
 70: (1, 1

In [32]:
Delta_ij_p = {i:[0]*ArcCountWithDummyArcs for i in Paths}

for p in Paths:
    for i in ArcDict:    
            if ArcDict[i] in PathDict[p]:
                Delta_ij_p[p][i] = 1
# Delta_ij_p

In [33]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (1, 10): 5,
 (1, 13): 6,
 (1, 14): 7,
 (1, 16): 8,
 (1, 17): 9,
 (1, 20): 10,
 (1, 22): 11,
 (1, 23): 12,
 (1, 25): 13,
 (1, 26): 14,
 (1, 27): 15,
 (1, 28): 16,
 (1, 32): 17,
 (1, 35): 18,
 (1, 37): 19,
 (1, 40): 20,
 (1, 41): 21,
 (1, 47): 22,
 (1, 48): 23,
 (1, 49): 24,
 (1, 50): 25,
 (1, 52): 26,
 (1, 54): 27,
 (1, 56): 28,
 (1, 59): 29,
 (1, 60): 30,
 (1, 65): 31,
 (1, 70): 32,
 (1, 71): 33,
 (1, 72): 34,
 (1, 77): 35,
 (1, 82): 36,
 (1, 84): 37,
 (1, 85): 38,
 (1, 87): 39,
 (1, 88): 40,
 (1, 89): 41,
 (1, 90): 42,
 (1, 96): 43,
 (1, 97): 44,
 (1, 101): 45,
 (1, 105): 46,
 (1, 110): 47,
 (1, 114): 48,
 (1, 116): 49,
 (1, 117): 50,
 (1, 118): 51,
 (1, 120): 52,
 (1, 121): 53,
 (1, 124): 54,
 (1, 125): 55,
 (1, 126): 56,
 (1, 127): 57,
 (1, 128): 58,
 (1, 130): 59,
 (1, 131): 60,
 (1, 133): 61,
 (1, 134): 62,
 (1, 135): 63,
 (1, 137): 64,
 (1, 138): 65,
 (1, 139): 66,
 (1, 140): 67,
 (1, 142): 68,
 (1, 144): 69,
 (1, 145):

In [35]:
def PathModel():
    m1 = gp.Model("PathModel")
    
    Flow = m1.addVars(Paths,Commodities,name="flow")

    NetworkCost = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            NetworkCost += B_k_Dict[k] * PathCost[p] * Flow[p,k]
        for p in Paths:
            if p not in PathSetForCommodity[k]:
                m1.addConstr(Flow[p,k]==0)
    
    m1.setObjective(NetworkCost, GRB.MINIMIZE)

    ArcFlow = [0]*ArcCountWithDummyArcs
    InfoCollected = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    ArcFlow[i] += B_k_Dict[k] * Flow[p,k]
            InfoCollected += PathInfo[p]* Flow[p,k]
    
    CapacityConstraint = [0]*ArcCountWithDummyArcs
    
#     for i in range(ArcCountWithDummyArcs):
    CapacityConstraint = m1.addConstrs((ArcFlow[i] <= CapacityDict[i] for i in range(ArcCountWithDummyArcs)),name="Cap")
        
    InfoThresholdConstraint = m1.addConstr(InfoCollected>=RequiredInfo,name="InfoT")
    
    FlowConstraint = m1.addConstrs(((Flow.sum('*',k))==1 for k in Commodities),name="FlowPart")
            
    m1.write("C:\PhD IE\Optimization Models for Large Networks\path.lp")
#     print(InfoThresholdConstraint)
    m1.optimize()
    
    if m1.Status == GRB.OPTIMAL:
        OptimalFlow = m1.getAttr('X',Flow)
        ReducedCosts = m1.getAttr('RC',Flow)
#         print(ReducedCosts)
        Pi_ij = m1.getAttr('Pi',CapacityConstraint)
        ThresholdDual = m1.getConstrByName("InfoT")
        W = ThresholdDual.Pi
        Sigma = m1.getAttr('Pi',FlowConstraint)
        for k in Commodities:
            for p in PathSetForCommodity[k]:
                if OptimalFlow[p,k] != 0:
                    print("Flow on path",p,"by commodity",k,"=",OptimalFlow[p,k]*B_k_Dict[k],"units.")
    return Pi_ij,W,Sigma

In [36]:
def PathModel2():
    m1 = gp.Model("PathModel")
#     m1.Params.Method = 0
    
    BigM = sum(B_k)*10 + RequiredInfo*ArcCountWithDummyArcs
    
    Flow = m1.addVars(Paths,Commodities,name="flow")
    Y = m1.addVars(ArcCountWithDummyArcs,Commodities,name="info")

    NetworkCost = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            NetworkCost += B_k_Dict[k] * PathCost[p] * Flow[p,k]
        for p in Paths:
            if p not in PathSetForCommodity[k]:
                m1.addConstr(Flow[p,k]==0)
    
    m1.setObjective(NetworkCost, GRB.MINIMIZE)

    ArcFlow = [0]*ArcCountWithDummyArcs
    InfoCollected = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    ArcFlow[i] += B_k_Dict[k] * Flow[p,k]
                    InfoCollected += InfoDict[i]* Y[i,k]
    
    CapacityConstraint = [0]*ArcCountWithDummyArcs
    
#     for i in range(ArcCountWithDummyArcs):
    CapacityConstraint = m1.addConstrs((ArcFlow[i] <= CapacityDict[i] for i in range(ArcCountWithDummyArcs)),name="Cap")
        
    InfoThresholdConstraint = m1.addConstr(InfoCollected>=RequiredInfo,name="InfoT")
    
    FlowCountOnArc = [0]*CommodityCount

    for k in Commodities:
        FlowCountOnArc[k-1] = [0]*ArcCountWithDummyArcs
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    FlowCountOnArc[k-1][i] += BigM * Flow[p,k]
    
    InfoOnlyWithFlowConstraint = m1.addConstrs(((FlowCountOnArc[k-1][i] - Y[i,k]) >= 0 for i in range(ArcCountWithDummyArcs)for k in Commodities),name="InfoWithFlow")
    
    FlowConstraint = m1.addConstrs(((Flow.sum('*',k))==1 for k in Commodities),name="FlowPart")

    InfoRepeatConstraint = m1.addConstrs(((Y.sum(i,'*'))<=1 for i in range(ArcCountWithDummyArcs)),name="InfoSum")
    
    m1.write("C:\PhD IE\Optimization Models for Large Networks\path2.lp")
#     print(InfoThresholdConstraint)
    m1.optimize()
    
    if m1.Status == GRB.OPTIMAL:
        OptimalFlow = m1.getAttr('X',Flow)
        ReducedCosts = m1.getAttr('RC',Flow)
#         print(ReducedCosts)
        Pi_ij = m1.getAttr('Pi',CapacityConstraint)
        ThresholdDual = m1.getConstrByName("InfoT")
        W = ThresholdDual.Pi
        Sigma = m1.getAttr('Pi',FlowConstraint)
        U = m1.getAttr('Pi',InfoOnlyWithFlowConstraint)
        V = m1.getAttr('Pi',InfoRepeatConstraint)
        for k in Commodities:
            for p in PathSetForCommodity[k]:
                if OptimalFlow[p,k] != 0:
                    print("Flow on path",p,"by commodity",k,"=",OptimalFlow[p,k]*B_k_Dict[k],"units.")
    return Pi_ij,W,Sigma,U,V

In [37]:
IterationCounter = 0
Pi_ijWithArcNumber,W,Sigma = PathModel()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9909 rows, 25 columns and 54 nonzeros
Model fingerprint: 0xe848b6a3
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+02, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 9909 rows and 25 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5145260e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.514526000e+06
Flow on path 0 by commodity 1 = 15.0 units.
Flow on path 1 by commodity 2 = 5.0 units.
Flow on path 2 by commodity 3 = 9.0 units.
Flow on path 3 by commodity 4 = 9.0 units.
Flow on path 4 by commodity 

In [38]:
Pi_ijWithArcNumber2, W2, Sigma2, U2, V2 = PathModel2()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 69207 rows, 49440 columns and 98889 nonzeros
Model fingerprint: 0x58e1231c
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+02, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 69207 rows and 49440 columns
Presolve time: 0.05s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5145260e+06   0.000000e+00   0.000000e+00      0s

Solved with dual simplex
Solved in 0 iterations and 0.11 seconds (0.03 work units)
Optimal objective  6.514526000e+06
Flow on path 0 by commodity 1 = 15.0 units.
Flow on path 1 by commodity 2 = 5.0 units.
Flow on path 2 by commodity 3 = 9.0 units.
Flow on path 3 by commodity 4

In [39]:
Pi_ij = {ArcDict[i]:0 for i in range(ArcCountWithDummyArcs)}

for i in range(ArcCountWithDummyArcs):
    Pi_ij[ArcDict[i]] = Pi_ijWithArcNumber[i]

# print("Pi=",Pi_ij)
# print("W=",W)
# print("Sigma=",Sigma)

In [40]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (1, 10): 5,
 (1, 13): 6,
 (1, 14): 7,
 (1, 16): 8,
 (1, 17): 9,
 (1, 20): 10,
 (1, 22): 11,
 (1, 23): 12,
 (1, 25): 13,
 (1, 26): 14,
 (1, 27): 15,
 (1, 28): 16,
 (1, 32): 17,
 (1, 35): 18,
 (1, 37): 19,
 (1, 40): 20,
 (1, 41): 21,
 (1, 47): 22,
 (1, 48): 23,
 (1, 49): 24,
 (1, 50): 25,
 (1, 52): 26,
 (1, 54): 27,
 (1, 56): 28,
 (1, 59): 29,
 (1, 60): 30,
 (1, 65): 31,
 (1, 70): 32,
 (1, 71): 33,
 (1, 72): 34,
 (1, 77): 35,
 (1, 82): 36,
 (1, 84): 37,
 (1, 85): 38,
 (1, 87): 39,
 (1, 88): 40,
 (1, 89): 41,
 (1, 90): 42,
 (1, 96): 43,
 (1, 97): 44,
 (1, 101): 45,
 (1, 105): 46,
 (1, 110): 47,
 (1, 114): 48,
 (1, 116): 49,
 (1, 117): 50,
 (1, 118): 51,
 (1, 120): 52,
 (1, 121): 53,
 (1, 124): 54,
 (1, 125): 55,
 (1, 126): 56,
 (1, 127): 57,
 (1, 128): 58,
 (1, 130): 59,
 (1, 131): 60,
 (1, 133): 61,
 (1, 134): 62,
 (1, 135): 63,
 (1, 137): 64,
 (1, 138): 65,
 (1, 139): 66,
 (1, 140): 67,
 (1, 142): 68,
 (1, 144): 69,
 (1, 145):

In [41]:
def ReducedCostCompute(Pi_ijWithArcNumber,W,Sigma):
    ReducedCost = {k:[] for k in Commodities}
    
    for k in Commodities:
        ReducedCost[k] = [0]*ArcCountWithDummyArcs
    
    for k in Commodities:
        for i in range(ArcCountWithDummyArcs):
#             for p in Paths:
                ReducedCost[k][i] += B_k_Dict[k]*CostDict[i] - B_k_Dict[k]*Pi_ijWithArcNumber[i] - W * InfoDict[i] #- Sigma[k]
            
    return ReducedCost

In [42]:
def ReducedCostCompute2(Pi_ijWithArcNumber, W, Sigma, U, V):
    ReducedCost = {k:[] for k in Commodities}
    BigM = sum(B_k)*10 + RequiredInfo*ArcCountWithDummyArcs
    
    for k in Commodities:
        ReducedCost[k] = [0]*ArcCountWithDummyArcs
    
    for k in Commodities:
        for i in range(ArcCountWithDummyArcs):
#             for p in Paths:
                ReducedCost[k][i] += B_k_Dict[k]*CostDict[i] - B_k_Dict[k]*Pi_ijWithArcNumber[i] - BigM * U[(i,k)] #- Sigma[k]
            
    return ReducedCost

In [43]:
def BellmanFord(src,k,ReducedCost,Pi_ijWithArcNumber,W,Sigma):

    dist = [float("Inf")] * NodeCount
    prev = [-1] * NodeCount
    dist[src-1] = 0
    prev[src-1] = 0

    for _ in range(NodeCount - 1):

        for u in N:
            for v in NodeNodeAdjacencyList[u]:
#                 print(u,v,k,ReversedArcDict[(u,v)])
                if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                    dist[v-1] = dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]]
                    prev[v-1] = u-1

    for u in N:
        for v in NodeNodeAdjacencyList[u]:
            if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                print("Graph contains negative weight cycle")
                return

    return dist, prev;
    

In [44]:
# Distance = [0]*NodeCount
# Prev = [0]*NodeCount


In [45]:
# Distance, Previous = BellmanFord(Origin[0],1)
# Destination

In [46]:
def MinReducedCostPath(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2):

    ReducedCost = ReducedCostCompute2(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2) 
#     print("Reduced Costs:",ReducedCost)
    SP = [0]*CommodityCount

    for k in Commodities:

        Distance, Previous = BellmanFord(Origin[k-1],k,ReducedCost,Pi_ijWithArcNumber2,W2,Sigma2)
        for i in range(len(Distance)):
            Distance[i] -= Sigma2[k] 
#         print(Distance,Previous)
        
        if Distance[Destination[k-1]-1] < 0:

            SPTemp = [Destination[k-1]]
            T = Destination[k-1]-1

            while T!=Origin[k-1]-1:

                T = Previous[T]
                SPTemp.insert(0,T+1)

            SP[k-1] = SPTemp

    return SP

In [47]:
def Pricing(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2):
    ShortestPath = MinReducedCostPath(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2)
#     for k in range(CommodityCount):
#         ShortestPath[k] -= Sigma[k+1]
#     print("SP,O,D:",ShortestPath,Origin,Destination,"\n")
    NewPathCtr = 0
    PathLength = len(Paths)
    for k in range(CommodityCount):
#         VarCount = 0
        if ShortestPath[k] != 0:

#             for i in range(len(NodesInPath)):
#                 if NodesInPath[i] != ShortestPath[k]:
                    NewPathCtr += 1
                    Paths.append(PathLength-1+NewPathCtr)
                    PathSetForCommodity[k+1].append(PathLength-1+NewPathCtr)
                    NodesInPath[PathLength-1+NewPathCtr] = ShortestPath[k]
                    PathDict[PathLength-1+NewPathCtr] = []
                    PathCost[PathLength-1+NewPathCtr] = 0
                    PathInfo[PathLength-1+NewPathCtr] = 0
                    Delta_ij_p[PathLength-1+NewPathCtr] = [0]*ArcCountWithDummyArcs

                    for i in range(len(ShortestPath[k])-1):
                        PathDict[PathLength-1+NewPathCtr].append((ShortestPath[k][i],ShortestPath[k][i+1]))
                        PathCost[PathLength-1+NewPathCtr] += CostDict[ReversedArcDict[(ShortestPath[k][i],ShortestPath[k][i+1])]]
                        PathInfo[PathLength-1+NewPathCtr] += InfoDict[ReversedArcDict[(ShortestPath[k][i],ShortestPath[k][i+1])]]

                    for p in range(len(Paths)):
                        for i in range(len(ArcDict)):
                            if ArcDict[i] in PathDict[p]:
                                Delta_ij_p[p][i]=1
    return NewPathCtr

In [48]:
StartTime = time.time()
PricingTime = 0
NewPathCtr = Pricing(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2)
# print(NewPathCtr, Paths)
IterationCounter = 1

while (NewPathCtr != 0) :#and IterationCounter < min(CommodityCount*CommodityCount,10*CommodityCount):
    IterationStartTime = time.time()
    IterationCounter += 1
    Pi_ijWithArcNumber2,W2,Sigma2,U2,V2 = PathModel2()
#     print(Pi_ijWithArcNumber)
#     print(W)
#     print(Sigma)
    PricingStartTime = time.time()
    NewPathCtr = Pricing(Pi_ijWithArcNumber2,W2,Sigma2,U2,V2)
    print("Iteration",IterationCounter, "had", NewPathCtr,"paths added.")
#     print(NewPathCtr, Paths)
    IterationEndTime = time.time()
    print("Iteration",IterationCounter,"took",IterationEndTime-IterationStartTime,"seconds.")
    print("Pricing problem took",IterationEndTime-PricingStartTime,"seconds.\n\n")
    PricingTime += IterationEndTime-PricingStartTime

print("\n\n\nNo new path added in iteration",IterationCounter,"\nSolution is optimal.")

EndTime = time.time()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 69227 rows, 49465 columns and 98966 nonzeros
Model fingerprint: 0xf697f679
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e+01, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 69212 rows and 49448 columns
Presolve time: 0.09s
Presolved: 15 rows, 17 columns, 41 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.100e+01
 Factor NZ  : 1.200e+02
 Factor Ops : 1.240e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.15555266e+06 -2.11918936e+03  3.38e+02 6.50e-02  2.81e+04     0s
   1  -1.29743945e+05 -3.52265882e

In [49]:
print("Time taken for the procedure to terminate:",EndTime-StartTime,"seconds.")
print("Time taken for pricing problem during the iterations:",PricingTime,"seconds.")

Time taken for the procedure to terminate: 9.46391224861145 seconds.
Time taken for pricing problem during the iterations: 4.269618511199951 seconds.
